In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import json
import requests
from pymongo import MongoClient
import numpy as np
from collections import defaultdict
import pickle

In [ ]:
from scrape_util import (load_json,
                                        write_json,
                                        selenium_instagram_login,
                                        setup_mongo_client)


In [ ]:
client, collection = setup_mongo_client('instascrape', 'influencer_likes')

# Making dictionary of top posts for top influencers

In [ ]:
hashtags_likes_dict = load_json('../data/hashtags_likes_dict.json')
with open('../data/influencer_graph.pkl') as myfile:
    top = pickle.load(myfile)

In [ ]:
# Collect posts for all top influencers
influencers_top = {}
for k,v in hashtags_likes_dict.iteritems():
    if k in top:
        influencers_top[k] = v

In [ ]:
# Collect most liked post for all top influencers
influencers_top_post = {}
for k,v in influencers_top.iteritems():
    influencers_top_post[k] = {'max_likes':0, 'post': None}
    for i in range(len(v['likes'])):
        if v['likes'][i] > influencers_top_post[k]['max_likes']:
            influencers_top_post[k]['max_likes'] = v['likes'][i]
            influencers_top_post[k]['post'] = v['posts'][i]

In [ ]:
top = top.nodes()

# Scrape Test

In [ ]:
#base_url = "https://www.instagram.com/graphql/query/?query_id=17864450716183058&variables=%7B%22shortcode%22%3A%22{}%22%2C%22first%22%3A100%7D"

In [ ]:
url_likes = "https://www.instagram.com/graphql/query/?query_id=17864450716183058&variables=%7B%22shortcode%22%3A%22{}%22%2C%22first%22%3A{}%7D"

In [ ]:
driver = webdriver.Firefox()

In [ ]:
selenium_instagram_login(driver, 'instagram_credentials.json')

In [ ]:
driver.get(url_likes.format(influencers_top_post[top[0]]['post'], str(influencers_top_post[top[0]]['max_likes'])))

In [ ]:
driver.find_element_by_id('tab-1').click()

In [ ]:
data = driver.find_element_by_css_selector('pre.data')

In [ ]:
my_json = json.loads(data.text)

In [ ]:
likes = my_json['data']['shortcode_media']['edge_liked_by']['edges']

In [ ]:
collection.insert_one({'id':top[0],'likes': likes})

In [ ]:
# Removed this record from mongo db

# SCRAPE THE REST!

In [ ]:
driver = webdriver.Firefox()
url_likes = "https://www.instagram.com/graphql/query/?query_id=17864450716183058&variables=%7B%22shortcode%22%3A%22{}%22%2C%22first%22%3A{}%7D"
selenium_instagram_login(driver, '../instagram_credentials.json')

In [ ]:
for i in range(len(top)):
    driver.get(url_likes.format(influencers_top_post[top[i]]['post'], str(influencers_top_post[top[i]]['max_likes'])))
    time.sleep(np.random.uniform(2,4))
    
    driver.find_element_by_id('tab-1').click()
    data = driver.find_element_by_css_selector('pre.data')
    my_json = json.loads(data.text)
    likes = my_json['data']['shortcode_media']['edge_liked_by']['edges']
    collection.insert_one({'id':top[i],'likes': likes})
    
    time.sleep(np.random.uniform(5,8))

# Saving the collected data

In [ ]:
cursor = collection.find({})
for x in cursor:
    user_id = x['id']
    likes = []
    for liker in x['likes']:
        likes.append(liker['node']['id'])
    influencers_top_post[user_id]['likes'] = likes

In [ ]:
write_json(influencers_top_post, 'data/influencers_top_posts_likes_1774.json')

# Checking the data

In [ ]:
for k,v in influencers_top_post.iteritems():
    print k, v['max_likes'], len(v['likes'])